In [83]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, Subset

In [85]:
class GraphLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, heads=1, concat=True, 
                 negative_slope=0.2, dropout=0, bias=True, inter_dim=-1,**kwargs):
        super(GraphLayer, self).__init__(aggr='add', **kwargs)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = dropout
        self.node_dim = 0
        self.__alpha__ = None
        self.lin = nn.Linear(in_channels, heads * out_channels, bias=False)

        self.att_i = nn.Parameter(torch.Tensor(1, heads, out_channels))
        self.att_j = nn.Parameter(torch.Tensor(1, heads, out_channels))
        self.att_em_i = nn.Parameter(torch.Tensor(1, heads, out_channels))
        self.att_em_j = nn.Parameter(torch.Tensor(1, heads, out_channels))

        if bias and concat:
            self.bias = nn.Parameter(torch.Tensor(heads*out_channels))
        elif bias and not concat:
            self.bias = nn.Parameter(torch.Tensor(out_channels))
        else:
            self.register_parameter('bias', None)


    def forward(self, x, edge_index, embedding, return_attention_weights=False):
        if torch.is_tensor(x):
            x = self.lin(x)
            x = (x, x)
        else:
            x = (self.lin(x[0]), self.lin(x[1]))

        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x[1].size(self.node_dim))
        out = self.propagate(edge_index, x=x, embedding=embedding, edges=edge_index, return_attention_weights=return_attention_weights)

        if self.concat:
            out = out.view(-1, self.heads * self.out_channels)
        else:
            out = out.mean(dim=1)

        if self.bias is not None:
            out = out + self.bias

        if return_attention_weights:
            alpha, self.__alpha__ = self.__alpha__, None
            return out, (edge_index, alpha)
        else:
            return out

    def message(self, x_i, x_j, edge_index_i, size_i, embedding, edges, return_attention_weights):
        x_i = x_i.view(-1, self.heads, self.out_channels)
        x_j = x_j.view(-1, self.heads, self.out_channels)

        if embedding is not None:
            embedding_i, embedding_j = embedding[edge_index_i], embedding[edges[0]]
            embedding_i = embedding_i.unsqueeze(1).repeat(1,self.heads,1)
            embedding_j = embedding_j.unsqueeze(1).repeat(1,self.heads,1)

            key_i = torch.cat((x_i, embedding_i), dim=-1)
            key_j = torch.cat((x_j, embedding_j), dim=-1)

        cat_att_i = torch.cat((self.att_i, self.att_em_i), dim=-1)
        cat_att_j = torch.cat((self.att_j, self.att_em_j), dim=-1)
        alpha = (key_i * cat_att_i).sum(-1) + (key_j * cat_att_j).sum(-1)
        alpha = alpha.view(-1, self.heads, 1)
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, edge_index_i, num_nodes = size_i)

        if return_attention_weights:
            self.__alpha__ = alpha

        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        
        return x_j * alpha.view(-1, self.heads, 1)

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__, self.in_channels, self.out_channels, self.heads)

    

class GNNLayer(nn.Module):
    def __init__(self, in_channel, out_channel, inter_dim=0, heads=1, node_num=100):
        super(GNNLayer, self).__init__()
        self.gnn = GraphLayer(in_channel, out_channel, inter_dim=inter_dim, heads=heads, concat=False)
        self.bn = nn.BatchNorm1d(out_channel)
        self.relu = nn.ReLU()
        self.leaky_relu = nn.LeakyReLU()

    def forward(self, x, edge_index, embedding=None, node_num=0):
        out, (new_edge_index, att_weight) = self.gnn(x, edge_index, embedding, return_attention_weights=True)
        self.att_weight_1 = att_weight
        self.edge_index_1 = new_edge_index
        out = self.bn(out)
        return self.relu(out)

In [70]:
class GraphConv(nn.Module):
    def __init__(self):
        super(GraphConv, self).__init__()
        in_feats = 36
        out_feats = 12
        
        self.gcn1 = GATConv(in_feats, out_feats)
        self.gcn2 = GATConv(out_feats, out_feats)
        self.linear = nn.Linear(out_feats, out_feats, bias = False)
    
        
    def forward(self, x, edge_index):
        x = x.permute(0, 2, 1)
        x = self.gcn1(x, edge_index)
        x = self.gcn2(x, edge_index)
        x = self.linear(x)
        x = x.permute(0, 2, 1)
        return x
        

In [71]:
model = GraphConv()
print(model)

GraphConv(
  (gcn1): GATConv(36, 12, heads=1)
  (gcn2): GATConv(12, 12, heads=1)
  (linear): Linear(in_features=12, out_features=12, bias=False)
)


In [72]:
def get_loaders(train_dataset, batch):
    train_dataloader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    return train_dataloader

In [73]:
train_df = pd.read_csv('C:/Users/harsh/OneDrive/Desktop/notebooks/calaveras_telemetry_multi.csv')
train_df = train_df.drop(columns = ['timestamp'])
print(train_df.shape)
print(train_df.head())

(10746, 6)
   requests_per_minute  request_duration  cpu_usage    memory        disk  \
0              30.6667         3157.2212     3.9381  676.4258  19798.1992   
1              30.6667         3157.2212     3.9381  676.4258  19798.1992   
2              25.3333         3208.7778     9.1459  668.0352  19798.3555   
3              25.3333         3208.7778     9.1459  668.0352  19798.3555   
4              25.3333         3208.7778     9.1459  668.0352  19798.3555   

   network_load  
0        0.0069  
1        0.0069  
2        0.3493  
3        0.3493  
4        0.3493  


In [74]:
def get_sequences(train_df, seq_len):
    train_df = train_df.values
    sequences = []
    for i in range(len(train_df)):
        j = i + seq_len
        if j < len(train_df):
            sequence = train_df[i:j, :]
            sequences.append(sequence)
    
    sequences = np.array(sequences)
    return sequences

sequences = get_sequences(train_df, 36)
sequences.shape

(10710, 36, 6)

In [75]:
train_dataloader = get_loaders(sequences, 32)

In [76]:
#random_batch = sequences[:32]
random_batch = sequences[:1]

In [77]:
random_batch.shape

(1, 36, 6)

In [78]:
def dense_graph(num_nodes):
    adj_matrix = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            if i!=j:
                adj_matrix.append([i, j])
    
    return torch.tensor(adj_matrix, dtype=torch.long).t().contiguous()

def node_feature(data):
    data = torch.Tensor(data)
    data = data.permute(0, 2, 1)
    return data

In [79]:
num_nodes = 6
edge_index = dense_graph(num_nodes)
x = node_feature(random_batch)

In [80]:
x[0].shape, edge_index.shape

(torch.Size([6, 36]), torch.Size([2, 30]))

In [81]:
out = model(x, edge_index)

AssertionError: Static graphs not supported in 'GATConv'